# 1. Importazione delle librerie



## Descrizione:
Iniziamo importando le librerie necessarie per il nostro progetto. Questo include:

torch per il lavoro con PyTorch,
transformers per il caricamento e utilizzo del modello T5,
pandas per la gestione dei dati,
tqdm per monitorare il progresso durante l'addestramento,
sklearn per suddividere il dataset in training e validation,
evaluate per il calcolo delle metriche, come il punteggio BLEU.

In [1]:
import torch
import json
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, RandomSampler
from transformers import T5TokenizerFast, T5ForConditionalGeneration
from torch.optim import Adam
import torch.nn as nn
from sklearn.model_selection import train_test_split
import evaluate 

# 2. Caricamento del modello e del tokenizer



## Descrizione:
Carichiamo il tokenizer e il modello pre-addestrato T5. Il modello t5-base è preconfigurato per il task di generazione di testo, e il tokenizer converte il testo in sequenze di token compatibili con il modello.

In [2]:
TOKENIZER = T5TokenizerFast.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)


# 3. Caricamento dei dati

## Descrizione:
Qui carichiamo i dati da un file JSON. Ogni oggetto JSON rappresenta un'istanza con domanda + contesto (input) e una risposta (output).

In [3]:
with open('new_t5_dataset.json', encoding='utf-8') as f:
    data = json.load(f)


# 4. Preparazione dei dati

## Descrizione:
La funzione prepare_data() elabora i dati JSON, separando la domanda (question) e il contesto (context) dal campo "input". La domanda e il contesto vengono estratti e organizzati in un formato utile per il modello.

In [4]:
def prepare_data(data):
    articles = []
    
    for item in data:
        input_text = item["input"]
        output_text = item["output"]
        
        question = input_text.split("Contesto:")[0].replace("Domanda:", "").strip()
        context = input_text.split("Contesto:")[1].strip()
        
        articles.append({"question": question, "context": context, "answer": output_text})

    return articles

data = prepare_data(data)


# 5. Creazione di un DataFrame

## Descrizione:
I dati elaborati vengono convertiti in un DataFrame di pandas per una gestione più semplice e per avere una panoramica dei dati.

In [5]:
df = pd.DataFrame(data)
print(df.head())


                                            question  \
0                                    Cos'è il MiCAR?   
1                         Perché è stato introdotto?   
2                              A cosa mira il MiCAR?   
3                                 Cosa sono gli EMT?   
4  Qual è l'approccio normativo di MiCAR riguardo...   

                                             context  \
0  Il MiCAR è il Regolamento sui Mercati delle Cr...   
1  Il co-legislatore europeo ha introdotto questa...   
2  Considerando che la politica monetaria della B...   
3  Gli EMT sono una delle tre categorie di cripto...   
4  L'approccio normativo di MiCAR non è quello di...   

                                              answer  
0  Il MiCAR è il Regolamento sui Mercati delle Cr...  
1  Il co-legislatore europeo ha introdotto questa...  
2  Considerando che la politica monetaria della B...  
3  Gli EMT sono una delle tre categorie di cripto...  
4  Nell'analizzare il regime giuridico degli EMT,..

# 6. Creazione del Dataset personalizzato



## Descrizione:
Definiamo un dataset personalizzato QA_Dataset che si occupa della tokenizzazione delle domande e risposte, gestendo anche il padding e la creazione dei mascheramenti necessari per l'addestramento del modello T5.

In [6]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']
        
    def __len__(self):
        return len(self.questions)
    
    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]
        
        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                            truncation=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length", 
                                          truncation=True, add_special_tokens=True)
        
        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100
        
        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }


# 7. Definizione dei parametri e del DataLoader

## Descrizione:
Inizializziamo vari parametri:

Q_LEN e T_LEN sono la lunghezza massima delle domande e risposte.
BATCH_SIZE determina il numero di esempi per batch.
DEVICE indica se utilizzare la CPU o la GPU.
Suddividiamo il dataset in training e validation e creiamo i dataloader che permetteranno di iterare sui dati durante l'addestramento.

In [7]:
Q_LEN = 256   # Question Length                                                                                                           
T_LEN = 64    # Target Length
BATCH_SIZE = 4
DEVICE ="cpu"  

train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, df, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)


## 8. Ottimizzazione e addestramento


## Descrizione:
Definiamo l'ottimizzatore (Adam) e iniziamo l'addestramento per 5 epoche (modificabili). Durante l'addestramento, monitoriamo la perdita di addestramento e di validazione.

In [8]:
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)

train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(5):  # Puoi modificare il numero di epoche
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(input_ids=input_ids, attention_mask=attention_mask, labels=labels,
                        decoder_attention_mask=decoder_attention_mask)

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1
    
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(input_ids=input_ids, attention_mask=attention_mask, labels=labels,
                        decoder_attention_mask=decoder_attention_mask)

        val_loss += outputs.loss.item()
        val_batch_count += 1
        
    print(f"{epoch+1}/{5} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")


Validation batches: 100%|██████████| 3/3 [00:08<00:00,  2.83s/it]


1/5 -> Train loss: 1.5395323417403481	Validation loss: 1.2986692388852437


Validation batches: 100%|██████████| 3/3 [00:08<00:00,  2.80s/it]


2/5 -> Train loss: 1.4565239927985452	Validation loss: 1.212925910949707


Validation batches: 100%|██████████| 3/3 [00:07<00:00,  2.65s/it]


3/5 -> Train loss: 1.3970363049796133	Validation loss: 1.145251366827223


Validation batches: 100%|██████████| 3/3 [00:09<00:00,  3.03s/it]


4/5 -> Train loss: 1.3448210968212648	Validation loss: 1.1019228001435597


Validation batches: 100%|██████████| 3/3 [00:08<00:00,  2.90s/it]

5/5 -> Train loss: 1.29934236461466	Validation loss: 1.0606210728486378


# 9. Salvataggio del modello

## Descrizione:
Dopo l'addestramento, salviamo il modello e il tokenizer per poterli riutilizzare in futuro.

In [9]:
MODEL.save_pretrained("qa_model")
TOKENIZER.save_pretrained("qa_model")


('qa_model\\tokenizer_config.json',
 'qa_model\\special_tokens_map.json',
 'qa_model\\spiece.model',
 'qa_model\\added_tokens.json',
 'qa_model\\tokenizer.json')

# 10. Funzione di previsione delle risposte

## Descrizione:
Infine, la funzione predict_answer() utilizza il modello addestrato per generare risposte a una domanda data un contesto. Se viene fornita una risposta di riferimento, calcola anche la metrica BLEU per valutare la qualità della risposta generata.

In [10]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)
    
    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)
  
    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)
    
    if ref_answer:
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer], references=[ref_answer])
    
        print("Context: \n", context)
        print("\n")
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer, 
            "Predicted Answer: ": predicted_answer, 
            "BLEU Score: ": score
        }
    else:
        return predicted_answer


In [11]:
answer=predict_answer("Il co-legislatore europeo ha introdotto questa nuova regolamentazione per diverse ragioni. Prima di tutto, era urgente colmare quella che era stata definita un'\"assenza di un quadro normativo stabile\" sia dall'ESMA (2019) che dall'EBA (2019). Le preoccupazioni sui rischi per i consumatori erano presenti poiché le cripto-attività non rientravano chiaramente nella definizione di strumenti finanziari (e, di conseguenza, nel campo di applicazione della MiFID II), lasciando un vuoto normativo. Inoltre, il legislatore mirava a evitare l'arbitraggio normativo tra i diversi Stati membri dell'UE: data la natura transfrontaliera degli asset basati su DLT, un approccio paese per paese sarebbe stato quasi certamente (e di fatto lo era) insufficiente. Infine, l'attività preparatoria da parte del legislatore è iniziata in un periodo in cui il numero di ICO è salito alle stelle. Le stablecoin non avevano garanzie sufficienti per assicurare la possibilità di convertirle in valuta legale. Queste stablecoin non rientravano nel controllo della BCE sull'emissione di denaro, ostacolando la sua capacità (e il suo dovere) di garantire la stabilità finanziaria e dei prezzi. L'introduzione del MiCAR è legata alla necessità di una regolamentazione chiara e uniforme per le cripto-attività nell'UE, evitando rischi per i consumatori e le disuguaglianze tra gli Stati membri.","Perché è stato introdotto il micar?","Perché è stato introdotto?")
print(answer)

Context: 
 Il co-legislatore europeo ha introdotto questa nuova regolamentazione per diverse ragioni. Prima di tutto, era urgente colmare quella che era stata definita un'"assenza di un quadro normativo stabile" sia dall'ESMA (2019) che dall'EBA (2019). Le preoccupazioni sui rischi per i consumatori erano presenti poiché le cripto-attività non rientravano chiaramente nella definizione di strumenti finanziari (e, di conseguenza, nel campo di applicazione della MiFID II), lasciando un vuoto normativo. Inoltre, il legislatore mirava a evitare l'arbitraggio normativo tra i diversi Stati membri dell'UE: data la natura transfrontaliera degli asset basati su DLT, un approccio paese per paese sarebbe stato quasi certamente (e di fatto lo era) insufficiente. Infine, l'attività preparatoria da parte del legislatore è iniziata in un periodo in cui il numero di ICO è salito alle stelle. Le stablecoin non avevano garanzie sufficienti per assicurare la possibilità di convertirle in valuta legale. Qu